In [152]:
#Import core libs
import re
import random

#Import non-core libs
import pandas as pd
import numpy as np

#Import Visualization libs

from bokeh.plotting import figure, output_notebook, show

In [153]:
#Load data to study

path_to_file = '~/Downloads/properati.csv'

data = pd.read_csv(path_to_file)

data.head(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [154]:
#Study Floor field current state ratio of nulls

total_records = data.shape[0]
total_nulls = data.floor.isnull().sum()

#print(total_nulls)
#print(total_records)

ratio = total_nulls / total_records * 100

print("Ratio of nulls on Floor Field: {} %".format(round(ratio, 2)))
print("Quantity of non-nulls records on Floor Field: {}.".format(total_records - total_nulls))


Ratio of nulls on Floor Field: 93.48 %
Quantity of non-nulls records on Floor Field: 7899.


In [155]:
#Understand Property Type composition

data.property_type.value_counts()

apartment    71065
house        40268
PH            5751
store         4136
Name: property_type, dtype: int64

In [187]:
#Study Floor field current state ratio of nulls for apartments only

apartments = data[data.property_type == "apartment"]

apartment_total_records = apartments.shape[0]
apartment_total_nulls = apartments.floor.isnull().sum()


apartment_ratio = apartment_total_nulls / apartment_total_records * 100

print("Ratio of nulls on Floor Field for Apartments: {} %".format(round(apartment_ratio, 2)))
print("Quantity of non-nulls records on Floor Field: {}.".format(apartment_total_records - apartment_total_nulls))

Ratio of nulls on Floor Field for Apartments: 90.82 %
Quantity of non-nulls records on Floor Field: 6521.


In [189]:
#Analyze how much information about the floor in apartments we can get from the Title


quantitative_samples = {
    "DEPTO EN TORRE 36. CUARTO PISO POR ESCALERA.",
    
}

qualitative_samples = {
    "Venta de Departamento 3 AMBIENTES en DUPLEX Flores, Capital FederalPosesion inmediataDepartamento alto, muy luminoso y con gran vista libre de edificios. Balcon terraza de 17,5 m2. Agua caliente central. Placard con interiores, bano y cocina completos. Dormitorio con vestidor. Apto profesional. Escritorio. Solarium y Parrilla en terraza. Instalacion de aire, falta el equipo. cochera opcional (u$s 17.000), fija en planta baja. -  - M.P CONSULTORA",
    
}

positional_words = {
    "primero": 1, 
    "primer": 1, 
    "segundo": 2, 
    "tercero": 3, 
    "cuarto": 4, 
    "quinto": 5, 
    "sexto": 6, 
    "septimo": 7, 
    "sept": 7, 
    "octavo": 8, 
    "noveno": 9,
    "decimo": 10, 
    "once": 11, 
    "doce": 12, 
}

words_for_floor = [
    "piso","pis", "iso","planta baja",
]

expression_for_floor = "({})".format(r"|".join(words_for_floor)) # piso|pis...


#Sort of fuzzy testing
def test_re_expression():
    
    r = range(32, 126) #really dirty set of ascii chars range
    a = [chr(random.choice(r)) for x in range()]
    
    scenarios = {
        "".join([c in random.choice]) : True
    }
    


re_for_floor = re.compile(expression_for_floor, re.IGNORECASE)

title_matched = apartments.title.apply(lambda title: bool(re_for_floor.match(title)))

apartments[title_matched].shape

#22 could be extracted from title, tho, it needs further matching to check if floor is already set.

(380, 26)

In [206]:
list(apartments.title.sample()) # each time will output a new title

['Departamento - Belgrano C']

In [217]:
#Analyze how much information about the floor in apartments we can get from the Description


#apartments[apartments.description.isnull()].description

apartments[apartments.description.apply(lambda desc: isinstance(desc,str)) == False]

def apply_re(description):
    try:
        return bool(re_for_floor.match(description))
    except TypeError:
        #print(description)
        return False

description_matched = apartments.description.apply(apply_re)

clean_1 = apartments[description_matched]
clean_2 = apartments[title_matched]

clean_2

#apartments[description_matched].shape

#367 could be extracted from description, tho, it needs further matching to check if floor is already set.

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
145,145,sell,apartment,Mar del Plata,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3430863.0,"-38.0038094,-57.5518491",-38.003809,...,183.0,NaN,2131.147541,NaN,5.0,NaN,http://www.properati.com.ar/15d87_venta_depart...,Piso de categoría con vista a Plaza Mitre.Rec...,Piso 5 ambientes vista a Plaza Mitre,https://thumbs4.properati.com/1/otzBbw0iMs1q3t...
149,149,sell,apartment,Mar del Plata,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3430863.0,"-37.9951681,-57.5563376",-37.995168,...,90.0,1545.454545,1888.888889,NaN,3.0,NaN,http://www.properati.com.ar/15d8m_venta_depart...,"Piso, por escalera, tres ambientes con bohardi...",Piso en Venta. Ambientes. Dormitorios. 110 m2....,https://thumbs4.properati.com/7/1MjWmEsyHmcj4i...
216,216,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5621457,-58.449657",-34.562146,...,150.0,3437.500000,3666.666667,NaN,4.0,NaN,http://www.properati.com.ar/15diu_venta_depart...,"Piso de categoría en Barrancas del Belgrano, e...",Piso Barrancas de Belgrano Venta 2 cocheras fi...,https://thumbs4.properati.com/3/83eGl5QZS6Glfm...
223,223,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5645474,-58.4488985",-34.564547,...,250.0,NaN,NaN,NaN,5.0,NaN,http://www.properati.com.ar/15djd_venta_depart...,Máxima categoría en ubicación y terminacionesA...,Piso Venta Amenities 2 cocheras fijas cubierta...,https://thumbs4.properati.com/6/0ZQNGbB95XZaug...
256,256,sell,apartment,Palermo,|Argentina|Capital Federal|Palermo|,Argentina,Capital Federal,3430234.0,"-34.5664787494,-58.4340748222",-34.566479,...,175.0,2514.285714,2514.285714,NaN,4.0,NaN,http://www.properati.com.ar/15dpv_venta_depart...,Excelente ubicación en el barrio de Cañitas4 A...,Piso 4 amb con dependencia y cochera en Venta ...,https://thumbs4.properati.com/7/_iZtVvjWtoiNtQ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118204,118204,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,...,NaN,3212.121212,NaN,NaN,NaN,6000.0,http://www.properati.com.ar/1cehq_venta_depart...,Belgrano Chico - Lindisimo piso de 4 ambientes...,Piso 4 amb- Dep - Cochera,https://thumbs4.properati.com/4/AlTb56wwDQo5Xy...
119296,119296,sell,apartment,Barrio Norte,|Argentina|Capital Federal|Barrio Norte|,Argentina,Capital Federal,3436109.0,"-34.5937427,-58.3929785",-34.593743,...,219.0,3246.753247,3424.657534,NaN,5.0,NaN,http://www.properati.com.ar/1cgg0_venta_depart...,Aire acondicionado.Molduras.Aberturas alemanas...,"Piso. 3 dormitorio(s) 1 suite con vest., 1 en ...",https://thumbs4.properati.com/8/J1Z-tPAGEQg60-...
119299,119299,sell,apartment,Barrio Norte,|Argentina|Capital Federal|Barrio Norte|,Argentina,Capital Federal,3436109.0,"-34.5858899,-58.3989103",-34.585890,...,260.0,3285.714286,3538.461538,NaN,5.0,NaN,http://www.properati.com.ar/1cgga_venta_depart...,"Departamento super luminoso, 260 metros + 20 m...",Piso espectacular TODO a nuevo como a estrenar!!,https://thumbs4.properati.com/9/tGL_5cSdhNhLKM...
119348,119348,sell,apartment,Balvanera,|Argentina|Capital Federal|Balvanera|,Argentina,Capital Federal,6693228.0,"-34.6052182255,-58.392127627",-34.605218,...,400.0,1750.000000,1750.000000,NaN,7.0,NaN,http://www.properati.com.ar/1cgic_venta_depart...,Departamento de 400 m2 sobre Av. Callao. Cuent...,Piso de 400 m2 sobre Callao,https://thumbs4.properati.com/7/KD4UpOYejH6f2q...


In [192]:
list(apartments.description.sample()) # each time will output a new title

['GARCIA ANDREU -Negocios Inmobiliarios-  Ofrece este departamento en Venta, situado en la calle Jujuy al 3100. Ubicado en el barrio de Pichincha a pocas cuadras del río Paraná, Bv. Oroño, Facultad de Medicina, Terminal de Ómnibus y a minutos del Microcentro. Zona que ofrece múltiples servicios e infraestructura, ideal para inversión.El Departamento es de 2 dormitorios con patio. Muy luminoso y de muy buena calidad constructiva. Está en construcción y ofrece amplia financiación en cuotas. En planta baja del edificio se ubican las cocheras. La terraza cuenta con parrillero.Las cocheras son opcionales y no están incluidas en el precio de cada unidad.']

In [208]:
properati_floor = data.loc[data.floor <= 20,['floor','price_usd_per_m2']]
properati_floor.groupby('floor').size()

floor
1.0     2080
2.0     1542
3.0      914
4.0      658
5.0      484
6.0      483
7.0      355
8.0      248
9.0      205
10.0     181
11.0     123
12.0     116
13.0      69
14.0      49
15.0      41
16.0      24
17.0      31
18.0      15
19.0      10
20.0       6
dtype: int64

In [209]:
mean_usd_per_m2_per_floor = data[data.floor <= 20].groupby('floor').mean()['price_usd_per_m2']

mean_usd_per_m2_per_floor.values

array([2144.43365524, 2253.39213176, 2308.44254245, 2289.76533436,
       2444.56836748, 2454.14083961, 2399.5750351 , 2432.09347446,
       2500.1365108 , 2731.67241272, 2759.32880944, 2839.6720894 ,
       2830.34645616, 3067.52216717, 3002.59554288, 2432.65229977,
       2993.59611563, 2214.43405753, 3516.63151663, 2049.23115079])

In [163]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure


floors = [str(x) for x in mean_usd_per_m2_per_floor.index]
counts = mean_usd_per_m2_per_floor.values

p = figure(x_range=floors, plot_height=250, title="Per Floor USD m2 Price Compairson",
           toolbar_location=None, tools="")

p.vbar(x=fruits, top=counts, width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0

output_notebook()


show(p)



Loading BokehJS ...

In [186]:
test_set = {
    '2 ambientes': True,
    '3 Ambientes': True,
    '4 AMBIENTES': True,
    '5 amx': False, # error a proposito,
    '10 amb': True
}

test_expression = re.compile(r'\d+\samb', re.IGNORECASE)

for phrase, expected_result in test_set.items():
    try:
        assert (test_expression.match(phrase) is not None) == expected_result
    except AssertionError as e:
        print('"{}": {}'.format(phrase, test_expression.match(phrase)))
        raise e



